# 数值稳定性和模型初始化

理解了正向传播与反向传播以后，我们来讨论一下深度学习模型的数值稳定性问题以及模型参数的初始化方法。深度模型有关数值稳定性的典型问题是衰减（vanishing）和爆炸（explosion）。


## 衰减和爆炸

当神经网络的层数较多时，模型的数值稳定性容易变差。假设一个层数为$L$的多层感知机的第$l$层$\boldsymbol{H}^{(l)}$的权重参数为$\boldsymbol{W}^{(l)}$，输出层$\boldsymbol{H}^{(L)}$的权重参数为$\boldsymbol{W}^{(L)}$。为了便于讨论，不考虑偏差参数，且设所有隐藏层的激活函数为恒等映射（identity mapping）$\phi(x) = x$。给定输入$\boldsymbol{X}$，多层感知机的第$l$层的输出$\boldsymbol{H}^{(l)} = \boldsymbol{X} \boldsymbol{W}^{(1)} \boldsymbol{W}^{(2)} \ldots \boldsymbol{W}^{(l)}$。此时，如果层数$l$较大，$\boldsymbol{H}^{(l)}$的计算可能会出现衰减或爆炸。举个例子，假设输入和所有层的权重参数都是标量，如权重参数为0.2和5，多层感知机的第30层输出为输入$\boldsymbol{X}$分别与$0.2^{30} \approx 1 \times 10^{-21}$（衰减）和$5^{30} \approx 9 \times 10^{20}$（爆炸）的乘积。类似地，当层数较多时，梯度的计算也更容易出现衰减或爆炸。

随着内容的不断深入，我们会在后面的章节进一步介绍深度学习的数值稳定性问题以及解决方法。


## 随机初始化模型参数

在神经网络中，通常需要随机初始化模型参数。下面我们来解释这样做的原因。

回顾[“多层感知机”](mlp.ipynb)一节图3.3描述的多层感知机。为了方便解释，假设输出层只保留一个输出单元$o_1$（删去$o_2$和$o_3$以及指向它们的箭头），且隐藏层使用相同的激活函数。如果将每个隐藏单元的参数都初始化为相等的值，那么在正向传播时每个隐藏单元将根据相同的输入计算出相同的值，并传递至输出层。在反向传播中，每个隐藏单元的参数梯度值相等。因此，这些参数在使用基于梯度的优化算法迭代后值依然相等。之后的迭代也是如此。在这种情况下，无论隐藏单元有多少，隐藏层本质上只有1个隐藏单元在发挥作用。因此，正如在前面的实验中所做的那样，我们通常将神经网络的模型参数，特别是权重参数，进行随机初始化。


### MXNet的默认随机初始化

随机初始化模型参数的方法有很多。在[“线性回归的简洁实现”](linear-regression-gluon.ipynb)一节中，我们使用`net.initialize(init.Normal(sigma=0.01))`使模型`net`的权重参数采用正态分布的随机初始化方式。如果不指定初始化方法，如`net.initialize()`，MXNet将使用默认的随机初始化方法：权重参数每个元素随机采样于-0.07到0.07之间的均匀分布，偏差参数全部清零。


### Xavier随机初始化

还有一种比较常用的随机初始化方法叫作Xavier随机初始化 [1]。
假设某全连接层的输入个数为$a$，输出个数为$b$，Xavier随机初始化将使该层中权重参数的每个元素都随机采样于均匀分布

$$U\left(-\sqrt{\frac{6}{a+b}}, \sqrt{\frac{6}{a+b}}\right).$$

它的设计主要考虑到，模型参数初始化后，每层输出的方差不该受该层输入个数影响，且每层梯度的方差也不该受该层输出个数影响。

## 小结

* 深度模型有关数值稳定性的典型问题是衰减和爆炸。当神经网络的层数较多时，模型的数值稳定性容易变差。
* 我们通常需要随机初始化神经网络的模型参数，如权重参数。


## 练习

* 有人说随机初始化模型参数是为了“打破对称性”。这里的“对称”应如何理解？
* 是否可以将线性回归或softmax回归中所有的权重参数都初始化为相同值？




## 参考文献

[1] Glorot, X., & Bengio, Y. (2010, March). Understanding the difficulty of training deep feedforward neural networks. In Proceedings of the thirteenth international conference on artificial intelligence and statistics (pp. 249-256).

## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/8052)

![](../img/qr_numerical-stability-and-init.svg)

## 个人小结与练习
### 个人小结
数值稳定性问题在训练中决定模型是否能够训练；训练基本方法是使用梯度下降的方法，而设计网络使其在训练中有良好的数值稳定性训练才可行。如果出现数值不稳定的情况，最容易出现的就是gradient explosion 与 vanishing。
- sofmax中的exponential（）还是如cross entropy中的log（）由于函数特性的问题，一些数值范围就容易出现数值不稳定的情况。为了防止这样的情况发生，常会对样本的数据做预处理的工作，如归一化操作等。
网络中的weight的随机初始化十分的重要，当weight初始化相同是；在训练反向传播更新weight阶段，他们初始值相同，这样各个nueron的gradient也是相同的，这样再复杂的网络实际上也就等同于只有一个hidden node。核心问题是反向传播计算gradient是在数值基础上来计算，当存在不同nueron但是其gradient相同时，效果上等同只有一个nueron。
- 随机初始化，mxnet中默认的方式是从-0.07到0.07范围以均匀分布来采样初始化。
- Xviar初始化，将输入与输出的相关尽一步降低；旨在消除输出层的方差不受到收入层个数的影响，每层的梯度也不受到输出成个数的影响。
 $$ U(-\sqrt{ \frac{6}{a+b}}, \sqrt{ \frac{6}{a+b}} )$$
- TODO. 对应还有kaming初始化，应用在网络中有relu activation时，具体见论文。
### 练习
- 初始化模型是在打破对称性，这里的对称就是在通过gradient decent方法学习是，forwardprop与backwardprop间不对称；对称也就是梯度gradient 相同；在网络中weights被初始化为相同时，参数就有了对称性，他们是identical的。
- 线性回归，单隐藏层的感知机；如果将参数相同初始化，隐藏层在训练时刻没有差异，此刻会降低feature表达能力，最后是否可能呢？因为做的是一个回归的操作，训练轮次的增加最后效果并不是很好，但是总是可行。
- softmax来说就不同了，目的是完成分类的操作；如果weight的初始化为相同的，训练中所有的hiddent nodes相同，最后的output 无法分出不同的类别；每个类别的feature都相同。